# Classification Project: Adult Census Income (UCI)

## 1. Problem Statement
We build a supervised **binary classification** model to predict whether an individual's annual income is **> $50K** (`income = 1`) or **<= $50K** (`income = 0`) using demographic and employment-related attributes.

## 2. UNSDG Alignment
This project aligns with **UN Sustainable Development Goal 8 (Decent Work and Economic Growth)** because it studies relationships between education, occupation, working hours, and income—key factors that inform policies for productive employment and improved living standards.

## 3. Dataset (required details)
**Dataset:** *Adult / Census Income* (UCI Machine Learning Repository)

- **Created / extracted:** Extracted from the **1994 U.S. Census database** by **Barry Becker**.
- **Donated / published on UCI:** **April 30, 1996**.
- **Access / source:** Downloaded as a CSV file from the UCI Machine Learning Repository.
- **Instances / features:** 48,842 rows and 14 input features (target not included in the 14).
- **Known issues:** Missing values exist for some categorical fields (often encoded as `?`).

**Primary reference:** UCI dataset page (Adult/Census Income).

## 4. Questions this dataset can answer (examples)
1. Which attributes (e.g., education level, occupation, working hours) are most associated with earning **> $50K**?
2. Are higher-income individuals older on average, or do they work more hours per week?
3. How does income vary by education and marital status?

---

> **Note:** The notebook uses a clean sklearn pipeline approach (imputation → encoding/scaling → model), so there is **no data leakage** between train and test.


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

FAST_MODE = True        # True = quicker experiments, False = full dataset run
SAMPLE_FRAC = 0.25

sns.set_theme(style="whitegrid")


## 3. Dataset (Adult / Census Income)

**Source:** UCI Machine Learning Repository (Adult / Census Income). The UCI page states the dataset was **donated on 30 April 1996**, and the extraction was done by **Barry Becker** from the **1994 US Census database**. (Commonly referenced contributors: Ron Kohavi and Barry Becker.)

**How the dataset was accessed in this notebook:** A local CSV file `adult.csv` (same schema as UCI Adult). If you do not have the file, download it from the UCI repository and rename to `adult.csv`.

### 3.1 Attributes (Features)
Each row represents one individual. The target label is `income`.

| Attribute | Type | Brief meaning |
|---|---:|---|
| age | numeric | Age in years |
| workclass | categorical | Employment type (e.g., Private, State-gov, Self-emp) |
| fnlwgt | numeric | Census sampling weight |
| education | categorical | Highest education level |
| education-num | numeric | Education level encoded as an integer |
| marital-status | categorical | Marital status |
| occupation | categorical | Occupation category |
| relationship | categorical | Relationship role in family |
| race | categorical | Race category |
| sex | categorical | Sex (Female/Male) |
| capital-gain | numeric | Capital gains |
| capital-loss | numeric | Capital losses |
| hours-per-week | numeric | Working hours per week |
| native-country | categorical | Country of origin |
| income (target) | categorical | `<=50K` or `>50K` |

### 3.2 Example questions the dataset can answer
1. Which attributes (education, occupation, hours/week, capital-gain, etc.) are most associated with earning **> $50K**?
2. How does **education level** affect the probability of earning **> $50K**?
3. Can we build a model that reliably predicts higher income while controlling for class imbalance?

### 3.3 Initial quality considerations
- The UCI Adult dataset includes **missing values** in some categorical columns (often encoded as `?`).
- The target is typically **imbalanced** (more `<=50K` than `>50K`), so we report **precision, recall, F1**, and also ROC-AUC / PR-AUC where useful.


**Interpretation **

This cell imports all libraries for data handling, visualization, machine learning, and evaluation. A fixed random seed is used so results are reproducible (same split and similar outcomes each run). FAST_MODE allows faster testing by sampling the dataset.

In [ ]:
df = pd.read_csv("adult.csv", na_values=["?"])
print("Shape:", df.shape)
display(df.head())

df = df.copy()

# Clean whitespace for text columns
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip()
    df.loc[df[c].str.lower() == "nan", c] = np.nan

# Ensure pandas NA becomes numpy nan (sklearn-friendly)
df = df.replace({pd.NA: np.nan})

TARGET = "income"
y_full = (df[TARGET] == ">50K").astype(int)
X_full = df.drop(columns=TARGET)


Interpretation:
Here we load the dataset and handle missing values (the dataset uses ? as missing). We also remove extra spaces from categorical values (like " Private"), which improves encoding quality. The target (income) is converted into binary classes: 1 for >50K and 0 for <=50K.

In [ ]:
# Initial data quality check on raw (uncleaned) data
print("Missing values before cleaning:")
display(df.isna().sum()[df.isna().sum() > 0])

print("Duplicate rows before cleaning:", df.duplicated().sum())


In [ ]:

# Data dictionary (feature descriptions)
data_dict = pd.DataFrame([
    ("age", "numeric (int)", "Age in years."),
    ("workclass", "categorical", "Employer type (e.g., Private, Self-emp, Government, etc.)."),
    ("fnlwgt", "numeric (int)", "Final sampling weight used by the Census Bureau (proxy for representation in population)."),
    ("education", "categorical", "Highest education level (string label)."),
    ("education-num", "numeric (int)", "Education level as an ordered numeric value."),
    ("marital-status", "categorical", "Marital status category."),
    ("occupation", "categorical", "Occupation category (e.g., Exec-managerial, Sales, etc.)."),
    ("relationship", "categorical", "Relationship / household role (e.g., Husband, Wife, Not-in-family)."),
    ("race", "categorical", "Race category."),
    ("sex", "binary categorical", "Sex (Female/Male)."),
    ("capital-gain", "numeric (int)", "Capital gains recorded for the year."),
    ("capital-loss", "numeric (int)", "Capital losses recorded for the year."),
    ("hours-per-week", "numeric (int)", "Hours worked per week."),
    ("native-country", "categorical", "Country of origin."),
    ("income (target)", "binary", "Target: >50K or <=50K annual income.")
], columns=["Attribute", "Type", "Description"])

display(data_dict)


In [ ]:
# Stratified sampling and quick data quality checks

# Use a smaller, representative subset of data when FAST_MODE is enabled.
# This speeds up experimentation while keeping the class ratio similar.
if FAST_MODE:
    tmp = X_full.copy()

    # Add target temporarily so we can sample within each class
    tmp["_y"] = y_full

    # Sample within each class to preserve the original class distribution
    tmp = tmp.groupby("_y", observed=True).sample(
        frac=SAMPLE_FRAC,
        random_state=RANDOM_STATE
    )

    # Separate features and target again after sampling
    y = tmp["_y"]
    X = tmp.drop(columns="_y")

    print(f"FAST_MODE=True → using {len(X):,} rows")
else:
    # Full dataset for final training and reporting
    X, y = X_full, y_full
    print(f"FAST_MODE=False → using full dataset: {len(X):,} rows")

# Human-readable labels for the target classes
labels = {0: "<=50K", 1: ">50K"}

# Show class distribution (helps spot imbalance before modeling)
display(pd.DataFrame({
    "Class": [labels[i] for i in y.value_counts().sort_index().index],
    "Count": y.value_counts().sort_index().values,
    "Percent": (y.value_counts(normalize=True).sort_index() * 100).round(2).values
}))

# Check missing values in X (should be 0 if you already dropped missing rows)
missing_pct = (X.isna().mean() * 100).sort_values(ascending=False)
display(missing_pct[missing_pct > 0].round(2).to_frame("Missing %"))

# Check duplicates in the original dataframe (before/after cleaning depends on where df is defined)
print("Duplicate rows in raw df:", df.duplicated().sum())


Interpretation:
This cell checks dataset quality. The target distribution shows class imbalance (more <=50K than >50K), so F1-score and recall become important, not just accuracy. We also measure missing values and duplicates to understand cleaning needs.

In [ ]:

# 4. Exploratory Data Analysis (EDA)

# Sanity check after cleaning
print("Total missing values in X:", X.isna().sum().sum())

# Quick structural summary
display(pd.DataFrame({
    "dtype": X.dtypes,
    "n_missing": X.isna().sum(),
    "missing_%": (X.isna().mean()*100).round(2),
    "n_unique": X.nunique()
}).sort_values("missing_%", ascending=False))

# Summary statistics (numeric)
display(X.describe(include=[np.number]).T)

# 4.1 Target distribution (class imbalance check)
plt.figure(figsize=(7,4))
sns.countplot(x=y.map(labels))
plt.title("Income Class Distribution")
plt.xlabel("Income")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 4.2 Missingness overview
miss_nonzero = missing_pct[missing_pct > 0].sort_values()
if len(miss_nonzero) > 0:
    plt.figure(figsize=(8,4))
    miss_nonzero.plot(kind="barh")
    plt.title("Missing Values by Column (%)")
    plt.xlabel("Missing %")
    plt.tight_layout()
    plt.show()

# Identify numeric vs categorical columns
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["int64", "float64"]).columns.tolist()
print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# 4.3 Age distribution + relationship with income
if "age" in X.columns:
    plt.figure(figsize=(7,4))
    sns.histplot(data=X, x="age", bins=30, kde=True)
    plt.title("Distribution of Age")
    plt.tight_layout()
    plt.show()

    tmp_plot = X[["age"]].copy()
    tmp_plot["income"] = y.map(labels)
    plt.figure(figsize=(7,4))
    sns.boxplot(data=tmp_plot, x="income", y="age")
    plt.title("Age by Income Group")
    plt.tight_layout()
    plt.show()

# 4.4 Hours-per-week and income
if "hours-per-week" in X.columns:
    plt.figure(figsize=(7,4))
    sns.histplot(data=X, x="hours-per-week", bins=30, kde=True)
    plt.title("Distribution of Hours Worked per Week")
    plt.tight_layout()
    plt.show()

    tmp_plot = X[["hours-per-week"]].copy()
    tmp_plot["income"] = y.map(labels)
    plt.figure(figsize=(7,4))
    sns.boxplot(data=tmp_plot, x="income", y="hours-per-week")
    plt.title("Hours-per-week by Income Group")
    plt.tight_layout()
    plt.show()

# 4.5 Education vs income (income rate)
if "education" in X.columns:
    edu_rate = pd.DataFrame({"education": X["education"], "y": y}).groupby("education")["y"].mean()
    edu_count = X["education"].value_counts()
    edu_plot = pd.DataFrame({"income_rate": edu_rate, "count": edu_count}).sort_values("count", ascending=False).head(12)
    plt.figure(figsize=(10,4))
    sns.barplot(x=edu_plot.index, y=edu_plot["income_rate"])
    plt.xticks(rotation=45, ha="right")
    plt.title("Income Rate (>50K) by Education (Top 12 by frequency)")
    plt.ylabel("P(income > 50K)")
    plt.tight_layout()
    plt.show()

# 4.6 Occupation vs income (top occupations)
if "occupation" in X.columns:
    occ_count = X["occupation"].value_counts().dropna()
    top_occ = occ_count.head(12).index
    occ_rate = pd.DataFrame({"occupation": X["occupation"], "y": y}).dropna().groupby("occupation")["y"].mean()
    occ_plot = occ_rate.loc[top_occ].sort_values(ascending=False)
    plt.figure(figsize=(10,4))
    sns.barplot(x=occ_plot.index, y=occ_plot.values)
    plt.xticks(rotation=45, ha="right")
    plt.title("Income Rate (>50K) by Occupation (Top 12 by frequency)")
    plt.ylabel("P(income > 50K)")
    plt.tight_layout()
    plt.show()

# 4.7 Capital gain is extremely right-skewed (many zeros). Use log1p for a clearer view.
if "capital-gain" in X.columns:
    cg = X["capital-gain"].fillna(0)
    plt.figure(figsize=(7,4))
    sns.histplot(np.log1p(cg), bins=40, kde=True)
    plt.title("log(1 + capital-gain) Distribution")
    plt.xlabel("log1p(capital-gain)")
    plt.tight_layout()
    plt.show()

# 4.8 Correlation heatmap (numeric features only)
if len(num_cols) >= 2:
    plt.figure(figsize=(7,5))
    sns.heatmap(X[num_cols].corr(numeric_only=True), annot=True, fmt=".2f", linewidths=0.5)
    plt.title("Correlation Heatmap (Numeric Features)")
    plt.tight_layout()
    plt.show()


### Key EDA Insights (what we learned)

- **Class imbalance:** The dataset contains more `<=50K` than `>50K`, so we prioritize **F1-score**, **recall**, and **ROC-AUC/PR-AUC** instead of only accuracy.
- **Missing values:** Missingness appears mainly in work-related categorical fields (e.g., *workclass*, *occupation*). We handle this with **imputation + one-hot encoding** in a preprocessing pipeline to avoid data leakage.
- **Age & hours worked:** Higher-income individuals generally show higher median *age* and higher *hours-per-week*.
- **Education & occupation:** Higher education levels and professional/managerial occupations have substantially higher `P(income > 50K)`.
- **Capital-gain:** Highly right-skewed (many zeros). The log1p view confirms a small group with very high gains.

These patterns justify (1) robust preprocessing, (2) models that can capture nonlinear interactions (e.g., Random Forest), and (3) evaluation beyond accuracy.


In [ ]:
# Split the dataset into training and testing sets
# Stratification ensures both income classes are represented proportionally
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# Print shapes to verify the split
print("Train/Test:", X_train.shape, X_test.shape)

# Preprocessing for numerical features
# Median imputation is robust to outliers, followed by standard scaling
numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Preprocessing for categorical features
# Missing values are filled with the most frequent category,
# then features are one-hot encoded
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combine numerical and categorical preprocessing steps
# ColumnTransformer applies the correct pipeline to each feature type
preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])


Interpretation:
We split into train and test sets using stratification so the income class ratio stays similar in both splits. Then we build a preprocessing pipeline: numeric features are imputed with median and scaled, while categorical features are imputed with the most frequent value and one-hot encoded.

## 5.1 Neural Network Model (MLPClassifier)

We implement a **Multi-Layer Perceptron (MLP)** as the neural network classifier.

**Architecture (chosen for tabular data):**
- Input: preprocessed numeric + one-hot categorical features
- Hidden layers: **(64, 32)** neurons
- Activation: **ReLU** (non-linear, works well for deep networks)
- Output layer: **sigmoid**-like probability via sklearn's internal logistic output for binary classification

**Training setup:**
- Loss: **log-loss (cross-entropy)** (default for classification in `MLPClassifier`)
- Optimizer: **Adam** (default, good general-purpose adaptive optimizer)
- Regularization: L2 controlled by `alpha`
- Early stopping: enabled to reduce overfitting

We evaluate using metrics suitable for imbalanced classes: **F1**, **recall**, and (when available) **ROC-AUC/PR-AUC**.


In [ ]:
# Baseline Models (2 Classical + 1 Neural Network) and Evaluation

# Extra metrics and curve utilities (useful when classes are not perfectly balanced)
from sklearn.metrics import roc_auc_score, average_precision_score, RocCurveDisplay, PrecisionRecallDisplay

def evaluate(model, name, show_curves=False):
    """Train the model and evaluate on the test set using multiple classification metrics."""

    # Fit the model on training data
    model.fit(X_train, y_train)

    # Predict class labels on the test set
    pred = model.predict(X_test)

    # Try to get predicted probabilities for ROC-AUC and PR-AUC
    # Not all models support predict_proba, so we check safely
    proba = None
    if hasattr(model, "predict_proba"):
        try:
            proba = model.predict_proba(X_test)[:, 1]   # probability of the positive class (>50K)
        except Exception:
            proba = None

    # Print a detailed classification report (precision/recall/F1 per class)
    print(f"\n{name}")
    print(classification_report(y_test, pred, target_names=["<=50K", ">50K"], zero_division=0))

    # Confusion matrix helps explain model errors clearly in viva
    cm = confusion_matrix(y_test, pred)
    plt.figure(figsize=(4.7, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(f"Confusion Matrix: {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.show()

    # Store core metrics for later comparison across models
    metrics = {
        "Model": name,
        "Accuracy": accuracy_score(y_test, pred),
        "Precision": precision_score(y_test, pred, zero_division=0),
        "Recall": recall_score(y_test, pred, zero_division=0),
        "F1": f1_score(y_test, pred, zero_division=0),
    }

    # ROC-AUC and PR-AUC require probabilities, so compute only if available
    # PR-AUC is especially informative when the positive class is smaller
    if proba is not None:
        metrics["ROC-AUC"] = roc_auc_score(y_test, proba)
        metrics["PR-AUC"] = average_precision_score(y_test, proba)

        # Optional: show ROC and Precision–Recall curves if requested
        if show_curves:
            RocCurveDisplay.from_predictions(y_test, proba)
            plt.title(f"ROC Curve: {name}")
            plt.tight_layout()
            plt.show()

            PrecisionRecallDisplay.from_predictions(y_test, proba)
            plt.title(f"Precision–Recall Curve: {name}")
            plt.tight_layout()
            plt.show()

    return metrics

# Logistic Regression baseline
# Good interpretable linear baseline; saga solver works well with larger feature spaces (after one-hot encoding)
logreg = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=3000, solver="saga", n_jobs=-1, random_state=RANDOM_STATE))
])

# Random Forest baseline
# Strong non-linear model that can capture feature interactions without heavy assumptions
rf = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=RANDOM_STATE))
])

# Neural network baseline (MLP)
# Early stopping helps avoid overfitting by stopping training when validation performance stops improving
mlp = Pipeline([
    ("prep", preprocess),
    ("clf", MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        early_stopping=True,
        alpha=1e-4,                # small L2 regularization to reduce overfitting
        learning_rate_init=1e-3,
        max_iter=60 if FAST_MODE else 150,   # fewer iterations in FAST_MODE to save time
        random_state=RANDOM_STATE
    ))
])

# Run evaluation for each baseline model and collect results in a list
baseline = []
baseline.append(evaluate(logreg, "Logistic Regression (Baseline)"))
baseline.append(evaluate(rf, "Random Forest (Baseline)"))
baseline.append(evaluate(mlp, "Neural Network MLP (Baseline)"))

# Convert results into a DataFrame for easy comparison
# Sorting by F1 is useful when class imbalance is present
baseline_df = pd.DataFrame(baseline).sort_values("F1", ascending=False)
display(baseline_df.round(4))


In [ ]:
# Use stratified k-fold cross-validation so each fold preserves class distribution
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Logistic Regression hyperparameter tuning
# Regularization strength, penalty type, and class weighting are tuned
# F1-score is used since the dataset is not perfectly balanced
gs_lr = GridSearchCV(
    logreg,
    {
        "clf__penalty": ["l2", "l1"],          # controls type of regularization
        "clf__C": [0.05, 0.1, 0.5, 1, 5],      # inverse of regularization strength
        "clf__class_weight": [None, "balanced"],  # helps handle class imbalance
        "clf__solver": ["saga"],               # supports both L1 and L2 penalties
    },
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=0
)

# Run grid search on training data
gs_lr.fit(X_train, y_train)

# Random Forest hyperparameter tuning
# Parameters mainly control model complexity and overfitting
gs_rf = GridSearchCV(
    rf,
    {
        "clf__n_estimators": [200, 400],       # number of trees in the forest
        "clf__max_depth": [None, 10, 20],      # limits tree depth to reduce overfitting
        "clf__min_samples_split": [2, 5, 10],  # minimum samples needed to split a node
        "clf__min_samples_leaf": [1, 2, 4],    # minimum samples in a leaf node
        "clf__class_weight": [None, "balanced"]  # balances classes during training
    },
    scoring="f1",
    cv=cv,
    n_jobs=-1,
    verbose=0
)

# Run grid search on training data
gs_rf.fit(X_train, y_train)

# Display best hyperparameters and corresponding cross-validation F1 scores
print("Best LR params:", gs_lr.best_params_)
print("Best LR CV F1:", round(gs_lr.best_score_, 4))
print("Best RF params:", gs_rf.best_params_)
print("Best RF CV F1:", round(gs_rf.best_score_, 4))

# Extract the best-performing models from grid search
best_lr = gs_lr.best_estimator_
best_rf = gs_rf.best_estimator_

# Evaluate tuned models on the test set
# ROC and Precision–Recall curves are shown for deeper performance analysis
tuned = []
tuned.append(evaluate(best_lr, "Logistic Regression (Tuned)", show_curves=True))
tuned.append(evaluate(best_rf, "Random Forest (Tuned)", show_curves=True))

# Compare tuned models using F1-score
display(pd.DataFrame(tuned).round(4).sort_values("F1", ascending=False))


Interpretation:
Hyperparameter tuning tries multiple settings using cross-validation and selects the best based on F1-score. This improves reliability because results are averaged across multiple folds rather than depending on a single train/test split.

In [ ]:
# Final Models (optimal hyperparameters + selected features) and comparative analysis

def count_selected_features(pipeline):
    """Count how many features remain after the feature-selection step."""
    # Fit the pipeline so the selector learns which features to keep
    pipeline.fit(X_train, y_train)

    # Try to access the feature selection step by name
    selector = pipeline.named_steps.get("select", None)
    if selector is None:
        return None

    # get_support() returns a boolean mask for selected features
    if hasattr(selector, "get_support"):
        return int(selector.get_support().sum())

    return None

# Logistic Regression with embedded feature selection using L1 regularization
# L1 pushes unimportant feature weights to zero, so SelectFromModel can drop them
lr_fs = Pipeline([
    ("prep", preprocess),
    ("select", SelectFromModel(
        LogisticRegression(
            penalty="l1",
            solver="saga",
            C=1.0,
            max_iter=5000,
            random_state=RANDOM_STATE
        )
    )),
    # Use the tuned Logistic Regression classifier from grid search
    ("clf", best_lr.named_steps["clf"])
])

# Random Forest with embedded feature selection using tree-based feature importance
rf_fs = Pipeline([
    ("prep", preprocess),
    ("select", SelectFromModel(
        RandomForestClassifier(
            n_estimators=400,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
    )),
    # Use the tuned Random Forest classifier from grid search
    ("clf", best_rf.named_steps["clf"])
])

final_results = []

# Train, cross-validate, and test both final models
for model, name in [(lr_fs, "Final Logistic Regression"), (rf_fs, "Final Random Forest")]:
    # Cross-validation score on training data gives a more stable estimate than one split
    cv_f1 = cross_val_score(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring="f1",
        n_jobs=-1
    ).mean()

    # Count selected features for the final comparison table
    n_feat = count_selected_features(model)

    # Evaluate on test set (and show ROC + PR curves)
    metrics = evaluate(model, name, show_curves=True)

    # Add feature count and CV score into the metrics dictionary
    metrics["Selected Features"] = n_feat
    metrics["CV F1"] = cv_f1

    final_results.append(metrics)

# Build final comparison table
final_table = pd.DataFrame(final_results)

# Arrange columns to match the assignment template as closely as possible
cols = ["Model", "Selected Features", "CV F1", "Accuracy", "Precision", "Recall", "F1", "ROC-AUC", "PR-AUC"]
final_table = final_table[[c for c in cols if c in final_table.columns]].round(4)

# Show the best model first based on F1-score
display(final_table.sort_values("F1", ascending=False))


## 7. Feature Selection (embedded methods)

We apply **embedded feature selection** because it selects features *during model training* and naturally matches our models:

1. **Logistic Regression (L1 penalty)**: L1 regularization drives unimportant coefficients to **exactly zero**, so we can keep only the non-zero (important) encoded features.
2. **Random Forest feature importance**: Tree-based models compute impurity-based importances; `SelectFromModel` retains the most informative features based on importance thresholding.

Why this is suitable here:
- The dataset contains many categorical variables; one-hot encoding creates **many sparse features**. Feature selection reduces dimensionality, can improve generalization, and makes the model easier to explain.

---
